## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [11]:
import pandas as pd

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [12]:
df = pd.read_csv("QueryResults.csv", names=["Date", "Language", "Post Count"])
df

,Date,Language,Post Count
0,m,TagName,NaN
1,2008-07-01 00:00:00,c#,3.0
2,2008-08-01 00:00:00,assembly,8.0
3,2008-08-01 00:00:00,c,83.0
4,2008-08-01 00:00:00,c#,505.0
...,...,...,...
2506,2023-08-01 00:00:00,php,1940.0
2507,2023-08-01 00:00:00,python,10861.0
2508,2023-08-01 00:00:00,r,2407.0
2509,2023-08-01 00:00:00,ruby,212.0


**Challenge**: Examine the first 5 rows and the last 5 rows of the dataframe

In [25]:
clean_df = df.dropna()
clean_df.tail()

,Date,Language,Post Count
2506,2023-08-01 00:00:00,php,1940.0
2507,2023-08-01 00:00:00,python,10861.0
2508,2023-08-01 00:00:00,r,2407.0
2509,2023-08-01 00:00:00,ruby,212.0
2510,2023-08-01 00:00:00,swift,1033.0


**Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [26]:
clean_df.shape

(2510, 3)

**Challenge**: Count the number of entries in each column of the dataframe

In [28]:
clean_df.count()

Date          2510
Language      2510
Post Count    2510
dtype: int64

**Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [36]:
aggregated_df = clean_df.groupby("Language")
aggregated_df.sum()

,Post Count
Language,
assembly,43193.0
c,398744.0
c#,1600155.0
c++,798073.0
delphi,51181.0
go,70522.0
java,1904287.0
javascript,2507957.0
perl,67804.0


Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [37]:
aggregated_df.count()

,Date,Post Count
Language,,
assembly,181,181
c,181,181
c#,182,182
c++,181,181
delphi,181,181
go,166,166
java,181,181
javascript,181,181
perl,181,181


## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [38]:
clean_df["Date"] = pd.to_datetime(clean_df["Date"])
clean_df.head()

C:\Users\Igor\AppData\Local\Temp\ipykernel_3496\2268027453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df["Date"] = pd.to_datetime(clean_df["Date"])


,Date,Language,Post Count
1,2008-07-01,c#,3.0
2,2008-08-01,assembly,8.0
3,2008-08-01,c,83.0
4,2008-08-01,c#,505.0
5,2008-08-01,c++,164.0


## Data Manipulation



**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 